In [ ]:
from modules.models import define_VAE, define_MM
from modules.variables import HDVariables
from keras.utils import plot_model
import h5py
import numpy as np
import keras

In [ ]:
from modules.data_prep import VAEdataPrep

### set variables

variables are saved to disk in directory "variables", if not any other directory is specified by the user.
This is to access variables easily when performing inference in a different notebook.

In [ ]:
hd_var = HDVariables()

hd_var.timesteps = 16
hd_var.features = 131
hd_var.latent_units = 30

hd_var.enc_units = 70
hd_var.dec_units = 70
hd_var.dropout = 0.1
hd_var.beta = 2
hd_var.learning_rate = 0.001
hd_var.epsilon_std = 1.

hd_var.mdn_seq_len = 10
hd_var.mdn_hidden_units = 50
hd_var.number_mixtures = 10

hd_var.batch_size = 64

### initialize VAE

In [ ]:
vae_full, _, _ = define_VAE(enc_units= hd_var.enc_units,
                            dec_units= hd_var.dec_units,
                            latent_units= hd_var.latent_units,
                            timesteps= hd_var.timesteps,
                            features= hd_var.features,
                            dropout= hd_var.dropout,
                            beta= hd_var.beta,
                            learning_rate= hd_var.learning_rate,
                            epsilon_std= hd_var.epsilon_std)

### Prepare the data (includes generator setup)
The data must be converted from arrays of integers to sliced, one hot encoded vectors.  
The encoded vectors are sliced according to the given 'timesteps' for one sequence.  
  
After being sliced, the vectors, hereby called 'bars', are saved as numpy arrays in directories with names that the user defines. Old directories with the same name will be deleted when calling the function. The user can check the progress with the option 'print_progress'.  
'print_progress' can be a number for selecting how many times to print the progress, or it can be True / False to print all or nothing of the progress.  
  
Both functions skips songs that are above the average bar number of [113 bars](https://www.statcrunch.com/5.0/viewreport.php?groupid=948&reportid=28647), and songs below or equal to 1 bar. 
  
### Define names for encoder and decoder input directories + fit_generator setup file

In [ ]:
hd_var.encoder_dir = "enc_in"
hd_var.decoder_dir = "dec_in"
hd_var.encoder_dir_songs = "song_enc_in"

hd_var.generator_IDs_file = "datasets/id_lists/generator_IDs.npy" # must be .npy

hd_var.songs_ID_file = "datasets/id_lists/song_IDs.npy" # for inference, creates lookup table for all songs

### Set correct dataset file

In [ ]:
dataset_file = "datasets/test_dataset.npy"
vae_data_prep = VAEdataPrep(timesteps = hd_var.timesteps,
                            features = hd_var.features)

In [ ]:
vae_data_prep.create_bars_dir(encoder_directory= hd_var.encoder_dir,
                            decoder_directory= hd_var.decoder_dir,
                            dataset_file= dataset_file,
                            generator_IDs_file= hd_var.generator_IDs_file,
                            print_progress= True)

In [ ]:
vae_data_prep.create_songs_dir(encoder_directory= hd_var.encoder_dir_songs,
                            inference_ID_file = hd_var.songs_ID_file,
                            dataset_file= dataset_file,
                            print_progress=True)

### setup GPU (voluntary)

In [ ]:
import os
import tensorflow as tf
import keras.backend as K

# Only for GPU use:
os.environ["CUDA_VISIBLE_DEVICES"]="1"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)

### setup the generator

In [ ]:
from modules.generators import VAEDataGenerator

# Parameters
params = {'dim': (hd_var.timesteps, hd_var.features),
          'enc_dir' : hd_var.encoder_dir,
          'dec_dir' : hd_var.decoder_dir,
          'batch_size': hd_var.batch_size,
          'shuffle': True}

ID_list_dict = {}

# Datasets
ID_list = np.load(hd_var.generator_IDs_file)
ID_list_dict["train"] = ID_list.item().get("train")
ID_list_dict["validation"] = ID_list.item().get("validation")

# Generators
training_generator = VAEDataGenerator(ID_list_dict['train'], **params)
validation_generator = VAEDataGenerator(ID_list_dict['validation'], **params)

# Train VAE

In [ ]:
from keras.callbacks import ModelCheckpoint,TensorBoard
from keras.callbacks import Callback

tb_dir = "tb"
filepath="weights/test_run-{epoch:02d}-{val_acc:.2f}.hdf5"

tensorboard = TensorBoard(log_dir = tb_dir, batch_size = hd_var.batch_size)
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint, tensorboard]


epochs = 1
num_training_samples = 10000
num_validation_samples = 1500
steps_per_epoch = num_training_samples // hd_var.batch_size
validation_steps = num_validation_samples // hd_var.batch_size


# Train model on dataset
vae_full.fit_generator(generator = training_generator,
                    validation_data = validation_generator,
                    steps_per_epoch = steps_per_epoch,
                    validation_steps = validation_steps,
                    use_multiprocessing = False,
                    epochs = epochs,
                    verbose = 2,
                    callbacks = callbacks_list)

# Create mixture model inputs

Initialize VAE encoder, and set the path for the best weights from the previous training. The encoder weights from this set of weights will be set to the encoder.  
  
To create mixture model inputs, all songs from directory of songs (created earlier) will be processed by the encoder, producing a latent vector (z) for each bar. All songs saved as sequences of z's.   

<img src="imgs/project_info/full_model.png">

<br>
<br>

<img src="imgs/project_info/z_creation.png">

### Set path for the best VAE weights

In [ ]:
vae_weights_path = "weights/" + "test_run-99-0.80.hdf5"

In [ ]:
# VAE encoder
_, vae_enc, _ = define_VAE(enc_units = hd_var.enc_units,
                        dec_units = hd_var.dec_units,
                        latent_units = hd_var.latent_units,
                        timesteps = hd_var.timesteps,
                        features = hd_var.features,
                        dropout = hd_var.dropout,
                        beta = hd_var.beta,
                        learning_rate = hd_var.learning_rate,
                        epsilon_std = hd_var.epsilon_std)

vae_full.load_weights(vae_weights_path)
print("Loaded VAE weights from disk")

# get all weights
all_weights = vae_full.get_weights()

# find encoder weights position
decoder_position = len(vae_enc.get_weights())
encoder_weights = all_weights[:decoder_position]

# set encoder weights
vae_enc.set_weights(encoder_weights)
print("Set VAE weights")

### Define name for z-dataset file

Prefereably put the z-dataset file in the folder *datasets*.  

In [ ]:
z_dataset_file = "datasets/z_dataset.h5"

After defining a name for the file, run the 'create_mm_data' from the MDNdataPrep module.  
  
The function show a progress bar, which usually gets a SynchronisationWarning, this doesn't cause a problem for the z-creation.

In [ ]:
from modules.data_prep import MDNdataPrep

mdn_data_prep = MDNdataPrep(timesteps = hd_var.timesteps,
                            features = hd_var.features,
                            z_dim = hd_var.latent_units,
                            z_dataset_file = z_dataset_file)

In [ ]:
mdn_data_prep.create_mm_data(vae_enc=vae_enc, encoder_dir_songs= hd_var.encoder_dir_songs)

### Define name for sliced z-dataset file, generator-IDs file and MDN sequence length

In [ ]:
hd_var.sliced_z_dataset_file = "datasets/sliced_z_dataset.h5"
MDNgenerator_IDs_file = "datasets/mdn_generator_IDs.npy" #must be numpy
hd_var.mdn_seq_len = 10

In [ ]:
mdn_data_prep.slice_z_data_for_mdn(hd_var.sliced_z_dataset_file,
                                   hd_var.MDNgenerator_IDs_file,
                                   hd_var.mdn_seq_len)

### check data, if you'd like

In [ ]:
import h5py
hf_mdn = h5py.File(hd_var.sliced_z_dataset_file, 'r')

print("Number of sliced z's: ", len(list(hf_mdn.keys()))/2)
print("Is the first time step of y the same as the second time step of x? \n" ,\
      list(hf_mdn.get("z_x_id-1"))[1] == list(hf_mdn.get("z_y_id-1"))[0])

hf_mdn.close()

### define MDN hidden units and number of mixtures

In [ ]:
hd_var.mdn_hidden_units = 50
hd_var.number_mixtures = 10

### initalize the trainable MDN

the function returns a MDN for inference also

In [ ]:
mm_full, _ = define_MM(seq_len = hd_var.mdn_seq_len,
                    vae_latent_units = hd_var.latent_units,
                    hidden_units = hd_var.mdn_hidden_units,
                    number_mixtures = hd_var.number_mixtures)

### setup the MDN generator

In [ ]:
from modules.generators import MDNDataGenerator

batch_size = 64

params = {'dim': (hd_var.mdn_seq_len, hd_var.latent_units),
          'batch_size': hd_var.batch_size,
          'dataset_path': hd_var.sliced_z_dataset_file,
          'shuffle': True}

ID_list_dict = {}
ID_list = np.load(MDNgenerator_IDs_file)

ID_list_dict["train"] = ID_list.item().get("train")
ID_list_dict["validation"] = ID_list.item().get("validation")

# Generators
training_generator = MDNDataGenerator(ID_list_dict['train'], **params)
validation_generator = MDNDataGenerator(ID_list_dict['validation'], **params)

# Train MDN

In [ ]:
epochs = 1
tb_log_dir = "tb/mdn"

num_training_samples = 10000
num_validation_samples = 1000
steps_per_epoch = int(num_training_samples / batch_size)
validation_steps = int(num_validation_samples / batch_size)

# Train model on dataset
filepath="weights/mdn_test_run-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorboard = keras.callbacks.TensorBoard(log_dir=tb_log_dir, batch_size=batch_size)

history = mm_full.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps,
                    use_multiprocessing=False,
                    epochs=epochs,verbose=2,callbacks=[keras.callbacks.TerminateOnNaN(),
                                                       checkpoint,
                                                       tensorboard])